## XBRL API in R
This sample R code queries the XBRL US API (https://xbrl.us/home/use/xbrl-api/)
### Enter your XBRL API credentials 
Enter  your credentials in the cell below as noted, then run all cells. 

In [9]:
#load required r libraries

library(httr)
library(jsonlite)


#insert your credentials between the quotation marks below

var.user <- "XBRL US Web account email"
var.pass <- "XBRL US password"
var.client <- "XBRL API Client ID"
var.secret <- "XBRL API Secret"

#authenticate for token

auth <- POST(
			url = "https://api.xbrl.us/oauth2/token",
            body = list(grant_type = "password",
						platform = "jupyterR",
                        client_id = var.client,
                        client_secret = var.secret,
						username = var.user,
						password = var.pass),
			encode = "form"
			)
var.access <- httr::content(auth)$access_token
var.token <- paste("Bearer", var.access)

  print(paste('Your access token', var.access, 'expires after 60 minutes. When it expires, re-run all cells to generate a new token.'))

[1] "Your access token 4a715711-58e8-4f7e-a7b0-7172764397b1 expires after 60 minutes. When it expires, re-run all cells to generate a new token."


### Make a query 
After the access token appears above, you can modify the query below, then use the **_Cell >> Run All Below_** menu option from the cell **immediately below this text** to run the entire query for results. 
  
**Once the initial data query is executed**, use *_Run All Below_* menu option at the highest point on this page where you modified the data query to get updated results. 
  
Refer to XBRL API documentation at https://xbrlus.github.io/xbrl-api/#/Facts/getFactDetails for other endpoints and parameters to filter and return.

In [10]:
var.host <- "https://api.xbrl.us/api/v1/"
var.endpoint <- "fact" # fact,report,concept,entity, etc. - some nesting is possible
var.params <- "entity.cik=0000789019&fact.has-dimensions=false&period.fiscal-period=Y" # endpoint-specific filters
var.concepts <- "AssetsCurrent,AssetsNoncurrent,Assets,LiabilitiesCurrent,LiabilitiesNoncurrent,Liabilities,StockholdersEquity,MinorityInterest,StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest,LiabilitiesAndStockholdersEquity" #concepts to look for in filings
var.fields <- "period.fiscal-year.sort(DESC),concept.local-name.sort(DESC),fact.value,entity.name" # details to be returned

var.url <- paste0(var.host, var.endpoint, "/search?", var.params, "&concept.local-name=", var.concepts, "&fields=", var.fields)

#request data

var.req <- httr::GET(var.url, httr::add_headers(Authorization = var.token))

#display results

var.json <- httr::content(var.req, as = "text")
var.data <- fromJSON(var.json)
print(var.url)
print(var.data)

[1] "https://api.xbrl.us/api/v1/fact/search?entity.cik=0000789019&fact.has-dimensions=false&period.fiscal-period=Y&concept.local-name=AssetsCurrent,AssetsNoncurrent,Assets,LiabilitiesCurrent,LiabilitiesNoncurrent,Liabilities,StockholdersEquity,MinorityInterest,StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest,LiabilitiesAndStockholdersEquity&fields=period.fiscal-year.sort(DESC),concept.local-name.sort(DESC),fact.value,entity.name"
$paging
$paging$limit
[1] 100

$paging$offset
[1] 0

$paging$count
[1] 100


$data
    period.fiscal-year               concept.local-name   fact.value
1                 2020               StockholdersEquity 118304000000
2                 2020               StockholdersEquity 118304000000
3                 2020               LiabilitiesCurrent  72310000000
4                 2020 LiabilitiesAndStockholdersEquity 301311000000
5                 2020                      Liabilities 183007000000
6                 2020                    Asset